This short example show how to get data from FMI Open Data multipointcoverage format. The format is used in INSPIRE specifications and is somewhat complex. Anyway, it's the most efficient way to get large amounts of data.

Here we fetch all observations from Finland during two days.

This example is for "old" format WFS2. You may try to use new WFS3 beta service as well. It's available in: http://beta.fmi.fi/data/3/wfs/sofp/

In [1]:
import requests
import datetime as dt
import xml.etree.ElementTree as ET
import numpy as np
import re

Required functions to get param names. Param keys are in the response document but longer names along with other metadata need to be fetched separately. 

In [2]:
def get_param_names(url):
    """ Get parameters metadata"""
    req = requests.get(url)
    params = {}
    
    if req.status_code == 200:
        xmlstring = req.content
        tree = ET.ElementTree(ET.fromstring(xmlstring))                
        for p in tree.iter(tag='{http://inspire.ec.europa.eu/schemas/omop/2.9}ObservableProperty'):
            params[p.get('{http://www.opengis.net/gml/3.2}id')] = p.find('{http://inspire.ec.europa.eu/schemas/omop/2.9}label').text
    return params        
    
def get_params(tree):
    """ Get parameters from response xml tree """
    
    retParams = []
    for el in tree.iter(tag='{http://www.opengis.net/om/2.0}observedProperty'):
        url = el.get('{http://www.w3.org/1999/xlink}href')
        params = re.findall(r"(?<=param=).*,.*(?=&)", url)[0].split(',')

        param_names = get_param_names(url)
        for p in params:
            retParams.append('{} ({})'.format(param_names[p], p))
                
    return retParams

Positions are in the separate element. Positions are listed as lat, lon, timestamp. 

In [3]:
def get_positions(tree):
    """ 
    Function to get times and coordinates from multipointcoverage answer
    """
    positions = []
    for el in tree.iter(tag='{http://www.opengis.net/gmlcov/1.0}positions'):
        pos = el.text.split()
        i = 0
        while len(pos) > 0:
            lat = float(pos.pop(0))
            lon = float(pos.pop(0))
            timestamp = int(pos.pop(0))
            positions.append([lat,lon,timestamp])
    return np.array(positions)

Get data. For longer periods we have to fetch data in the loop

In [4]:
url = 'http://opendata.fmi.fi/wfs'
starttime = dt.datetime.strptime('2010-01-01', "%Y-%m-%d")
endtime = dt.datetime.strptime('2010-01-03', "%Y-%m-%d")
daystep = 1

start = starttime
end = start + dt.timedelta(days=daystep)
if end > endtime: end = endtime
    
while end <= endtime and start < end:
    startStr = start.strftime('%Y-%m-%d')
    endStr = end.strftime('%Y-%m-%d')
    
    # Get data
    payload = {
        'request': 'getFeature',
        'storedquery_id': 'fmi::observations::weather::multipointcoverage',
        'bbox': '19,59,35,75',
        'starttime': startStr,
        'endtime': endStr,    
    }
    r = requests.get(url, params=payload)
    
    # Construct XML tree
    tree = ET.ElementTree(ET.fromstring(r.content))

    # Get geospatial and temporal positions of data elements
    positions = get_positions(tree)

    # Extract data from XML tree
    d = []
    for el in tree.iter(tag='{http://www.opengis.net/gml/3.2}doubleOrNilReasonTupleList'):
        for pos in el.text.strip().split("\n"):
            d.append(pos.strip().split(' '))
    
    # Assign data values to positions
    junk = np.append(positions, np.array(d), axis=1)
    try:
        data = np.append(data, junk, axis=0)
    except NameError:
        data = junk
    
    print('Time interval {} - {} provided {} rows'.format(startStr, endStr, junk.shape[0]))
    
    start = end
    end = start + dt.timedelta(days=daystep)
    if end > endtime: end = endtime

print('Done fetching data. Final dimensions of the result: {}'.format(data.shape))

Time interval 2010-01-01 - 2010-01-02 provided 23406 rows
Time interval 2010-01-02 - 2010-01-03 provided 23375 rows
Done fetching data. Final dimensions of the result: (46781, 16)


Get params from the last XML tree element (they don't change over time)

In [5]:
params = get_params(tree)

Finally you can do whatever you want with the data. Here we just print some example.

In [6]:
print('Params: {}'.format(params))
print(data[0:2])

Params: ['Air temperature (t2m)', 'Wind speed (ws_10min)', 'Gust speed (wg_10min)', 'Wind direction (wd_10min)', 'Relative humidity (rh)', 'Dew-point temperature (td)', 'Precipitation amount (r_1h)', 'Precipitation intensity (ri_10min)', 'Snow depth (snow_aws)', 'Pressure (msl) (p_sea)', 'Horizontal visibility (vis)', 'Cloud amount (n_man)', 'Present weather (auto) (wawa)']
[['60.12467' '19.90362' '1262304000.0' '-11.7' '2.6' '2.6' '360.0' '90.0'
  '-13.0' 'NaN' 'NaN' 'NaN' '1003.0' '35000.0' '8.0' '0.0']
 ['60.12467' '19.90362' '1262304600.0' '-11.5' '2.6' '3.1' '20.0' '91.0'
  '-12.7' 'NaN' 'NaN' 'NaN' '1003.0' '35000.0' '8.0' '0.0']]
